In [1]:
import tensorflow as tf
import tensorflow._api.v2.experimental.tensorrt
from tensorflow.python.compiler.tensorrt import trt_convert as trt
import tensorflow.python.compiler.tensorrt.trt_convert as con
#tgc = trt.TrtGraphConverterV2()
params = tensorflow._api.v2.experimental.tensorrt.ConversionParams(
        precision_mode='FP16',
        # Set this to a large enough number so it can cache all the engines.
        maximum_cached_engines=16)
converter = tensorflow._api.v2.experimental.tensorrt.Converter(
        input_saved_model_dir="saved_model.pb", conversion_params=params)
converter.convert()
'''
input_saved_model_dir = "./model/saved_model.h5" 
output_saved_model_dir = ""
converter = trt.TrtGraphConverter(input_saved_model_dir=input_saved_model_dir)
converter.convert()
converter.save(output_saved_model_dir)'''

ERROR:tensorflow:Tensorflow needs to be built with TensorRT support enabled to allow TF-TRT to operate.


RuntimeError: Tensorflow has not been built with TensorRT support.

In [ ]:
import tensorflow as tf

def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = tf.graph_util.convert_variables_to_constants(
            session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph

In [5]:
import tensorflow as tf
from tensorflow.python.compiler.tensorrt import trt_convert as trt

input_saved_model_dir = "" 
output_saved_model_dir = ""

params = tf.experimental.tensorrt.ConversionParams(
    precision_mode='FP16')
converter = tf.experimental.tensorrt.Converter(
    input_saved_model_dir="", conversion_params=params)
#converter.convert()
#converter.save(output_saved_model_dir)


converter = trt.TrtGraphConverter(input_saved_model_dir=input_saved_model_dir)
converter.convert()
converter.save(output_saved_model_dir)

ERROR:tensorflow:Tensorflow needs to be built with TensorRT support enabled to allow TF-TRT to operate.


RuntimeError: Tensorflow has not been built with TensorRT support.

In [1]:
import tensorflow as tf
import tensorrt as trt

# Load the HDF5 model
model = tf.keras.models.load_model('')

# Convert the model to a TensorRT optimized model
trt_model = trt.tensorrt.TRTModule()
trt_model.from_keras(model)

# Save the TensorRT model to an engine file
trt_model.save('model.engine')

OSError: No file or directory found at 

In [ ]:
'''
Usage:
    freeze_model.py --model="mymodel.h5" --output="frozen_model.pb"

Note:
    This requires that TensorRT is setup correctly. For more instructions, take a look at
    https://docs.nvidia.com/deeplearning/sdk/tensorrt-install-guide/index.html
'''
import os

from docopt import docopt
import json
from pathlib import Path
import tensorflow as tf

args = docopt(__doc__)
in_model = os.path.expanduser(args['--model'])
output = os.path.expanduser(args['--output'])
output_path = Path(output)
output_meta = Path('%s/%s.metadata' % (output_path.parent.as_posix(), output_path.stem))


# Reset session
tf.keras.backend.clear_session()
tf.keras.backend.set_learning_phase(0)

model = tf.keras.models.load_model(in_model, compile=False)
session = tf.keras.backend.get_session()

input_names = sorted([layer.op.name for layer in model.inputs])
output_names = sorted([layer.op.name for layer in model.outputs])

# Store additional information in metadata, useful when creating a TensorRT network
meta = {'input_names': input_names, 'output_names': output_names}

graph = session.graph

# Freeze Graph
with graph.as_default():
    # Convert variables to constants
    graph_frozen = tf.compat.v1.graph_util.convert_variables_to_constants(session, graph.as_graph_def(), output_names)
    # Remove training nodes
    graph_frozen = tf.compat.v1.graph_util.remove_training_nodes(graph_frozen)
    with open(output, 'wb') as output_file, open(output_meta.as_posix(), 'w') as meta_file:
        output_file.write(graph_frozen.SerializeToString())
        meta_file.write(json.dumps(meta))

    print ('Inputs = [%s], Outputs = [%s]' % (input_names, output_names))
    print ('Writing metadata to %s' % output_meta.as_posix())
    print ('To convert use: \n   `convert-to-uff %s`' % (output))

In [1]:
import tensorflow as tf
import onnx
import onnx_tf

model = tf.saved_model.load("")

onnx_model = onnx_tf.backend.prepare(model, target_opset=7)
onnx.save(onnx_model, "saved_model.onnx")

ImportError: cannot import name 'builder' from 'google.protobuf.internal' (c:\Users\bhavy\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\internal\__init__.py)